<a href="https://colab.research.google.com/github/pandaxin8/COVID-19/blob/master/labs/wk8/07_kernel_regression_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kernel Methods

###### COMP4670/8600 - Statistical Machine Learning - Tutorial

In this lab, we will understand the relationship between linear regression and kernel regression by an excercise in predicting the median value of houses in Boston in two ways: using a quadratic basis and using an equivalent kernel function.

### Assumed knowledge
- Linear regression (week 2)
- Kernel methods (week 7 lectures)


### Resources
- Lecture recording and notes (week 6 lectures)
- Bishop, Chapter 6 (Kernel Methods), Introduction and Section 1 (Dual Representations)
- Murphy's Machine Learning A Probabilistic Perspective, Chapter 14 (Kernels), 14.2 (Kernel Functions) and 14.4.3 (Kernelised Ridge Regression)


### After this lab, you should be comfortable with:
- Applying machine learning techniques with a non-linear basis
- Using kernel methods instead of a new basis
- Evaluating when using a kernel method would or would not be sensible

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\dotprod}[2]{\langle #1, #2 \rangle}$

Setting up the environment

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

%matplotlib inline

## Loading the dataset
In this lab, we will use the same [dataset](https://machlearn.gitlab.io/sml2019/tutorials/02-dataset.csv) as in week 2, which describes the price of housing in Boston (see [description](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)). 
We aim to predict the value of a home from other factors.
In this dataset, each row represents the data of one house. The first column is the value of the house, which is the target to predict. The remaining columns are features, which has been normalised to be in the range $[-1, 1]$. The corresponding labels of columns are

```'medv', 'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'```

Download the dataset. Read in the data using ```np.loadtxt``` with the optional argument ```delimiter=','```, as our data is comma separated rather than space separated. Remove the column containing the binary variable ```'chas'```.

Check that the data is as expected using ```print()```. It should have 506 rows (examples) and 13 columns (1 label and 12 features). Check that this is the case. 

Hint: use  assert.

In [2]:
names = ['medv', 'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat']

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
loaded_data = np.loadtxt('/content/drive/MyDrive/SML/labs/wk8/07-dataset.csv', delimiter=',')

In [5]:
#loaded_data = np.loadtxt('07-dataset.csv', delimiter=',')

# remove chas
column_idxes = list(range(len(names)))
chas_idx = names.index('chas')
wanted_columns = list(column_idxes)
wanted_columns.remove(chas_idx)
data = loaded_data[:,wanted_columns]
data_names = list(names)
data_names.remove('chas')

print(data_names)
print(np.array_str(data[:5], precision=3))
assert data.shape == (506,13)

['medv', 'crim', 'zn', 'indus', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat']
[[24.    -1.    -0.64  -0.864 -0.37   0.155  0.283 -0.462 -1.    -0.584
  -0.426  1.    -0.821]
 [21.    -1.    -1.    -0.515 -0.654  0.096  0.565 -0.302 -0.913 -0.79
   0.106  1.    -0.591]
 [34.    -1.    -1.    -0.515 -0.654  0.389  0.199 -0.302 -0.913 -0.79
   0.106  0.979 -0.873]
 [33.    -0.999 -1.    -0.874 -0.7    0.317 -0.116 -0.103 -0.826 -0.866
   0.298  0.989 -0.933]
 [36.    -0.999 -1.    -0.874 -0.7    0.374  0.057 -0.103 -0.826 -0.866
   0.298  1.    -0.801]]


## Part 0: Introduction

We breifly review some concepts from the class: you may also want to look at the Resources list.

### Kernel Functions and Basis Functions
Kernel functions can be understood as a way to measure the similarity of two objects $x$ and $x'$ in some space $\mathcal{X}$,
$$
k: \mathcal{X} \times \mathcal{X} \to  \mathbb{R}
$$
and especially, we mostly wish to consider positive definite kernels, which we will just call kernels. One powerful property of such kernels is that theses are exactly the functions which can be expressed as the inner product of some basis function transformation:
$$
k: \mathcal{X} \times \mathcal{X} \to  \mathbb{R} \: \text{is a kernel}  \iff k(x,x') = \langle \phi_{k}(x), \phi_{k}(x')  \rangle_{\mathcal{V}}
$$
for some basis function $\phi_k : \mathcal{X}\to \mathcal{V}$, and inner product $\langle . \: , \: \rangle_\mathcal{V}$. For our purposes, $\mathcal{X}$ and $\mathcal{V}$ may be simply real spaces $\mathbb{R}^{d}$, $\mathbb{R}^{d'}$, and $\langle x,y\rangle_\mathbb{R^{d'}}=x^T y$. 

The important aspect is the correspondence between some kernel function and a basis function. Fix our standard inner product on real spaces as above. Given a kernel, we can find a basis function whose inner product equals the kernel; given a basis function, we can define a kernel via the inner product:
$$
k \to \phi_k \: s.t. \: \phi_k(x)^T \phi_k(y) = k(x,y)
$$
and
$$
\phi \to k_{\phi} \: s.t. \: k_{\phi}(x,y) :=  \phi(x)^T \phi(y)
$$

In Part 2 of the lab, you will be given a kernel and required to prove that it is expressible via some basis fucntion.

#### Quesion 0.1: While all (positive definite) kernels are expressible as an inner product of basis functions in some space, not all are expressible in finite dimensional space. Give an example of a kernel which cannot be expressed via a finite dimensional basis function.

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

My response:
------------------


Gaussian kernel is an example of a kernel that can't be expressed via a finite dimensional basis function.

 The Gaussian kernel is defined as:

$k(x, y) = exp(-||x - y||^2 / (2σ^2))$

where x and y are input vectors, ||x - y||^2 is the squared Euclidean distance between them, and σ is a positive parameter called the bandwidth.

- The Gaussian kernel maps the input data into an infinite-dimensional space, meaning that it cannot be represented by a finite-dimensional basis function
- This is because the Gaussian kernel is related to the Fourier transform of a Gaussian function, which results in an infinite series of basis functions
- The infinite-dimensional nature of the Gaussian kernel is what allows it to be a powerful and flexible tool for modeling complex and non-linear relationships in data.


### Kernel Methods
#### Question 0.2: Explain the basic idea of kernel methods to a classmate. Your answer should cover:
- In what sense is a kernel method non-parametric?
- In contrast to a parametric model like standard linear regression, how does a kernel method use the training data to eventually make a prediction for a new datapoint? Outline the general structure of both a paramatric model and a kernel model.
- In the kernel methods context, what is the Gram matrix $\mathbf{K}$, and what is an entry in it in terms of the kernel function $k$ and dataset $\{x_i\}_{n=1}^{N}$?

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

My response:
-----------------------
1. Basic idea of kernel methods:

Kernel methods are a class of non-parametric machine learning algorithms that operate by mapping input data to a higher-dimensional space, often implicitly, using a kernel function. The kernel function measures the similarity between data points in this higher-dimensional space, allowing the algorithm to find non-linear relationships in the data more easily.

2. Non-parametric nature of kernel methods:

A kernel method is considered non-parametric because it doesn't assume a specific functional form or a fixed number of parameters for the relationship between inputs and outputs. Instead, it directly relies on the training data and the chosen kernel function to define the model. This flexibility allows kernel methods to adapt to a wide range of data distributions and model complex relationships that might not be well-represented by a parametric model with a fixed functional form.

3. Parametric vs. kernel model prediction:

In a parametric model like standard linear regression, the relationship between inputs (features) and outputs (labels) is defined by a fixed functional form with a finite number of parameters. For example, in linear regression, the relationship is given by y = wx + b, where w and b are the parameters (weights and bias) to be learned from the training data. To make a prediction for a new data point, the model applies the learned parameters to the input features using the assumed functional form.

In contrast, a kernel method uses the training data more directly. For a new data point, the kernel method computes the similarity between the new point and each of the training data points using the kernel function. The prediction is then made by combining the similarities with the training outputs (labels) in a weighted manner. This approach allows kernel methods to capture non-linear relationships without explicitly specifying a functional form.

4. Gram matrix and its entries:

In the context of kernel methods, the Gram matrix (denoted as 𝚿 or K) is a matrix that contains the pairwise similarities between all the data points in the dataset, as computed by the kernel function k. The entries of the Gram matrix are defined as:

$K(i, j) = k(x_i, x_j)$

where $x_i$ and $x_j$ are the i-th and j-th data points in the dataset ${x_i}_{n=1}^{N}$, and k is the kernel function. The Gram matrix plays a crucial role in kernel methods, as it encapsulates the similarity information used to make predictions for new data points.

### Kernel Methods, Linear Regression and Dual Representation
This lab compares a familiar technique, regularised linear regression with a basis function, to the method of kernel regression. 

We recall that our linear regression optimises the parameter $\mathbf{w}$ of our model $f_{w}(x')=\mathbf{w}^T \phi(x')$ on transformed datapoints $x'$ best minimises the regularised sum-of-squares loss
$$J(\mathbf{w}) = \frac{1}{2} \sum_{n=1}^N(\mathbf{w}^T \phi(\mathbf{x_n}) - t_n)^2 + \frac{\lambda}{2} \mathbf{w}^T\mathbf{w}, \:\:\:\:(A)$$
Now, ultimately we will perform kernel regression by a function
$$
f(x') = \mathbf{k}(x')^T (\mathbf{K} + \lambda \mathbf{I})^{-1} \mathbf{t}
$$
where $\mathbf{k}(x')$ is the vector of kernel outputs $k(x', x_i)$ for each $x_i$ in the training data. First however, we wish to derive an expression for the loss as a function of $\mathbf{a}$ in terms of $\mathbf{a}$ and the Gram matrix $\mathbf{K}$:
$$J(\mathbf{a}) = \frac{1}{2} \mathbf{a}^T \mathbf{KKa} - \mathbf{a}^T \mathbf{Kt} + \frac{1}{2}\mathbf{t}^T \mathbf{t} + \frac{\lambda}{2} \mathbf{a}^T\mathbf{Ka} , \:\:\:\:(B)$$
where $\mathbf{K=\Phi \Phi}^T$ with elements $K_{nm} = \phi(\mathbf{x_n})^T\phi(\mathbf{x_m})$, and $\mathbf{a}$ is the vector with entries $a_n = -\frac{1}{\lambda} (\mathbf{w}^T \phi(x_n)-t_n)$ for each $(x_n, t_n)$ in the training data.

#### Question 0.3: Derive (B) from (A) (don't spend too long on this, you may leave it and continue with the lab)

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

My response:
- Come back to this

## Overview of Lab

### Part 0.5: Linear Regression Review
We breifly review linear regression with basis functions, and you implement code to find the analytic solution to $\mathbf{w}$

### Part 1: Linear Regression with Quadratic Basis Function
Implement a quadratic basis function $\phi(x)$ and perform linear regression.

### Part 2: Linear Regression with a Corresponding Kernel
Show that a given kernel function $k(x,y)$ is equivalent to the quadratic basis in Part 1. Implement a calculation of $k(x,y)$ across datasets via matrix operations, and perform kernelised linear regression.

### Part 3: Comparison and Reflection
Discuss the above linear regression and kernelised linear regression.

### Part 3.5: Other Kernels (Optional)
Implement a new kernel $k'(x,y)$ of your choice, and see it used in kernelised linear regression.



## Part 0.5: Linear Regression Review

First remind yourself of what linear regression is by our implemention of linear regression with regularisation on the Boston dataset. Use 80% of the available data for training the model using maximum likelihood with regularisation (assuming Gaussian noise). The rest of the data is allocated to the test set.

Report the root mean squared error (RMSE) for the training set and the test set. We'll compare the results with the linear regression with basis function, and the linear regression with kernel function later on.

**TODO**: Implement the analytic solution of $\frac{\partial J(\mathbf{w})}{\partial\mathbf{w}}=0$ in the function  ```w_ml_regularised(Phi, t, l)```, where ```l``` stands for $\lambda$. Note that $$\Phi=
\begin{pmatrix}
    \phi(\mathbf{x_1})^T \\
    \phi(\mathbf{x_2})^T \\
    \vdots \\
    \phi(\mathbf{x_n})^T \\
\end{pmatrix},$$
where $\phi(\mathbf{x_i})$ denotes the feature vector for the $i$-th datapoint, and $n$ denotes the total number of datapoints.

In [6]:
def w_ml_regularised(Phi, t, l):
    """Produce the analytic solution of w given input features and labels."""
    
    # Compute the regularized term: lambda * identity matrix
    reg_term = l * np.identity(Phi.shape[1])
    
    # Compute the inverse of (Phi^T * Phi + lambda * I)
    inv_term = np.linalg.inv(np.dot(Phi.T, Phi) + reg_term)
    
    # Compute the solution for w
    w = np.dot(inv_term, np.dot(Phi.T, t))
    
    return w

def split_data(data, train_size):
    """Randomly split data into two groups. The first group is a fifth of the data."""
    np.random.seed(1)
    N = len(data)
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:int(train_size * N)]
    test_idx = idx[int(train_size * N):]

    # Assume label is in the first column
    X_train = data[train_idx, 1:]
    t_train = data[train_idx, 0]
    X_test = data[test_idx, 1:]
    t_test = data[test_idx, 0]
    
    return X_train, t_train, X_test, t_test

def rmse(X_train, t_train, X_test, t_test, w):
    """Return the RMSE for training and test sets"""
    N_train = len(X_train)
    N_test = len(X_test)

    # Training set error
    t_train_pred = np.dot(X_train, w)
    rmse_train = np.linalg.norm(t_train_pred - t_train) / np.sqrt(N_train)

    # Test set error
    t_test_pred = np.dot(X_test, w)
    rmse_test = np.linalg.norm(t_test_pred - t_test) / np.sqrt(N_test)

    return rmse_train, rmse_test

def lr(X, l, split_rate):
    """Return RMSE for the training set and the test set
    
    Parameters
    ------------------------------------------------------
    X: numpy matrix, whole dataset 
    l: float, regularisation parameter
    split_rate: int, the percent of training dataset
    
    Returns
    ------------------------------------------------------
    train_rmse: float, RMSE for training set
    test_rmse: float, RMSE for testing set
    """
    X0 = np.ones((data.shape[0], 1))
    X = np.hstack((X, X0))

    X_train, t_train, X_test, t_test = split_data(X, split_rate)
    # alternatively: use train_test_split
    #X_train, X_test, t_train, t_test = train_test_split(X[:,1:], X[:, 0], test_size = 1 - split_rate, random_state = 42)
    w_reg = w_ml_regularised(X_train, t_train,l)

    train_rmse, test_rmse = rmse(X_train, t_train, X_test, t_test, w_reg)
    return train_rmse, test_rmse

train_rmse, test_rmse = lr(data, 1.1, 0.8)
print("Regression: RMSE with regularisation: Train {:.6f}, Test {:.6f}".format(train_rmse, test_rmse))
print("Expected value: Train 4.74, Test 4.89")

Regression: RMSE with regularisation: Train 4.741045, Test 4.891587
Expected value: Train 4.74, Test 4.89


## Part 1: Linear regression with quadratic basis function

Let $X \in \RR^{n \times d}$ be the data matrix containing n datapoints $\mathbf{x_i} \in \RR^{d}$. We can choose to train and test using the raw data $X$ as the input to our model as the above method. Alternatively, we could use $$\Phi= [\mathbf{\phi(x_1)}, \mathbf{\phi(x_2)}, ..., \mathbf{\phi(x_n)}]^T \in \RR^{n \times m},$$ where $\mathbf{\phi(x_i)}$ is some transformation of $\mathbf{x_i}$, m is the dimension of $\mathbf{\phi(x_i)}$.

For this lab, write $\mathbf{x_i} = (x_{i,1},\dots,x_{i,d})$. Let
$$
\phi(\mathbf{x_i}) = (x_{i,1}^2, x_{i,2}^2, \ldots, x_{i,d}^2, \sqrt{2}x_{i,1} x_{i,2}, \ldots, \sqrt{2}x_{i,d-1}x_{i,d}, \sqrt{2}x_{i,1}, \ldots, \sqrt{2}x_{i,d}, 1).
$$
Note that $\phi(\mathbf{x_i})$ is all quadratic functions of elements of $\mathbf{x_i}$ and 1 (The $\sqrt{2}$ coefficients are for normalisation for later in the lab).
We say that we are using a *quadratic basis function*.

Train and test the data with ```quadratic_lr```, report the RMSE for the training set and the test set.

**TODO**:  
1. write a function called ```phi_quadratic``` with a single datapoint $\mathbf{x_i}$ as input and the quadratic basis function $\phi(\mathbf{x_i})$ as output.  
2. write a function called ```feature_map``` with raw data matrix $X$ as input and $\Phi$ as output by using ```phi_quadratic``` for each datapoint. 
3. in function ```quadratic_lr```, make use of previous functions and give the analytic solution for $\mathbf{w}$.

In [8]:
def phi_quadratic(x):
    """Compute phi(x) for a single training example using quadratic basis function.
       
       1. Compute quadratic basis function for a single data point 'x' of shape (d,)
       2. First computes quadratic terms by squaring each element of 'x'
       3. Then computes the interaction terms as pairwise products of elems of 'x', scaled by 'sqrt(2)'
       4. Then linear terms are computed by scaling 'x' by 'sqrt(2)'
       5. Then constant term is added as array of one element [1]
       6. Computed terms are concatenated into a single feature vector 'feat' of shape (m,)
       7. Function returns the computed feature vector 'feat'
    
    """
    
    d = len(x)
    quadratic_terms = x**2
    interaction_terms = np.sqrt(2) * np.array([x[i] * x[j] for i in range(d) for j in range(i+1, d)])
    linear_terms = np.sqrt(2) * x
    constant_term = np.array([1])
    
    feat = np.concatenate([quadratic_terms, interaction_terms, linear_terms, constant_term])
    return feat # (m,)

def feature_map(X):
    """Return the matrix of the feature map.
      X.shape = (n,d)

      1. Compute feature map for dataset 'X' of shape (n,d) using quadratic basis fx
      2. Apply 'phi_quadratic' fx to each row (data point) of 'X'
      3. Resulting matrix 'Phi' has shape (n,m) where m is dimension of transformed feature vector
    """
    
    Phi = np.apply_along_axis(phi_quadratic, 1, X)
    return Phi # (n,m)

def quadratic_lr(X, l, split_rate):
    """Return RMSE for the training set and the test set
    
    Parameters
    ------------------------------------------------------
    X: numpy matrix, whole dataset 
    l: float, regularisation parameter
    split_rate: int, the percent of training dataset
    
    Returns
    ------------------------------------------------------
    train_rmse: float, RMSE for training set
    test_rmse: float, RMSE for testing set

    How it works
    1. Performs LR with quadratic basis function on dataset 'X' of shape (n,d)
      - regularisation param 'l' and split rate to divide dataset in train and test
    2. Splits dataset into train and test
    3. Computes feature maps 'Phi_train' and 'Phi_test'
    4. Optimal weights are computed using 'w_ml_regularised' (part 0.5)
    5. Compute RMSE using feature maps and optimal weights
    """
    
    X_train, t_train, X_test, t_test = split_data(X, split_rate)

    Phi_train = feature_map(X_train)
    w_reg = w_ml_regularised(Phi_train, t_train, l)

    train_rmse, test_rmse = rmse(feature_map(X_train), t_train, feature_map(X_test), t_test, w_reg)
    return train_rmse, test_rmse

train_rmse, test_rmse = quadratic_lr(data, 1.1, 0.8)
print("Quadratic basis: RMSE with regularisation: Train {:.6f}, Test {:.6f}".format(train_rmse, test_rmse))
print("Expected value: Train 2.79, Test 3.34")

Quadratic basis: RMSE with regularisation: Train 2.790632, Test 3.339125
Expected value: Train 2.79, Test 3.34


## Part 2: Linear regression with a  Corresponding Kernel

### Computing a basis transformation as an inner product

Define $k(\mathbf{x,y}) = (\dotprod{\mathbf{x}}{\mathbf{y}} + 1)^2$, where $\mathbf{x,y} \in \mathbb{R}^2$. One way to verify $k(\mathbf{x,y})$ is a kernel function is to write this as an inner product of a verctor valued function evaluated at $\mathbf{x}$ and $\mathbf{y}$. That is, show we have $k(\mathbf{x}, \mathbf{y}) = \dotprod{\phi(\mathbf{x})}{\phi(\mathbf{y})}$ and specify what is $\phi(\mathbf{x}), \phi(\mathbf{y})$.

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

My response
-------------
Given the kernel function $k(\mathbf{x}, \mathbf{y}) = (\langle\mathbf{x}, \mathbf{y}\rangle + 1)^2$, we need to find a feature map $\phi(\cdot)$ such that $k(\mathbf{x}, \mathbf{y}) = \langle\phi(\mathbf{x}), \phi(\mathbf{y})\rangle$.

Let $\mathbf{x} = (x_1, x_2)$ and $\mathbf{y} = (y_1, y_2)$, then the kernel function is:

Then convince yourself that, for $X \in \RR^{n \times d}, Y \in \RR^{m \times d}$, then $K = (X Y^T  + 1)^2 \in \RR^{n \times m}$ (addition and square term are element-wise) contains elements as kernel function between each pair of datapoints of X and Y,
$$K_{ij} = \phi(\mathbf{x_i})^T \phi(\mathbf{y_j}) = k(\mathbf{x_i}, \mathbf{y_j}).$$

### Kernelised Regression
**TODO**:
Write the function ```kernel_quadratic``` which takes $X$, $Y$  as input and $K$ as output.

My response
----------
We are given matrices $X \in \mathbb{R}^{n \times d}$ and $Y \in \mathbb{R}^{m \times d}$, and we want to compute the Gram matrix $K = (XY^T + 1)^2 \in \mathbb{R}^{n \times m}$, where the addition and squaring operations are element-wise.

Recall that the kernel function is defined as $k(\mathbf{x}, \mathbf{y}) = (\langle\mathbf{x}, \mathbf{y}\rangle + 1)^2$. Our goal is to show that $K_{ij} = k(\mathbf{x_i}, \mathbf{y_j}) = \langle\phi(\mathbf{x_i}), \phi(\mathbf{y_j})\rangle$.

First, let's compute the matrix product $XY^T$. The element at position $(i, j)$ in the resulting matrix is given by:

In [9]:
def kernel_quadratic(X, Y):
    # Compute the product XY^T
    XYT = np.dot(X, Y.T)
    
    # Add 1 element-wise to the resulting matrix
    XYT_plus_1 = XYT + 1
    
    # Square the elements element-wise
    K = np.power(XYT_plus_1, 2)
    
    return K

**TODO**:
Complete the function ```kernelised_lr``` with raw data matrix $X$ as input and root mean squared error (RMSE) for the training set and the test set as output. Inside of the function, make use of ```kernel_quadratic``` to apply dual representation, and calculate the predicted labels for training data and testing data repectively.

In [10]:
def kernelised_lr(X, l, split_rate):
    """Return RMSE for the training set and the test set
    
    Parameters
    ------------------------------------------------------
    X: numpy matrix, whole dataset 
    l: float, regularisation parameter
    split_rate: int, the percent of training dataset
    
    Returns
    ------------------------------------------------------
    rmse_train: float, RMSE for training set
    rmse_test: float, RMSE for testing set
    """
    
    X_train, t_train, X_test, t_test = split_data(X, split_rate)
    N_train = X_train.shape[0]
    N_test = X_test.shape[0]
    
    # Compute the kernel matrices for the training and test sets
    K_train = kernel_quadratic(X_train, X_train)
    K_test = kernel_quadratic(X_test, X_train)
    
    # Compute the dual solution alpha
    alpha = np.linalg.solve(K_train + l * np.eye(N_train), t_train)
    
    # Training set error
    t_train_pred = np.dot(K_train, alpha)
    rmse_train = np.linalg.norm(t_train_pred - t_train) / np.sqrt(N_train)

    # Test set error
    t_test_pred = np.dot(K_test, alpha)
    rmse_test = np.linalg.norm(t_test_pred - t_test) / np.sqrt(N_test)

    return rmse_train, rmse_test

train_rmse, test_rmse = kernelised_lr(data, 1.1, 0.8)
print("Kernelised Regression: RMSE with regularisation: Train {:.6f}, Test {:.6f}".format(train_rmse, test_rmse))
print("Expected value: Train 2.79, Test 3.33")

Kernelised Regression: RMSE with regularisation: Train 2.790632, Test 3.339125
Expected value: Train 2.79, Test 3.33


**Explanation of dual solution alpha**

In the context of kernel-based methods, the dual solution, often denoted as $\boldsymbol{\alpha}$, is a vector of coefficients that arises when we reformulate the original linear regression problem using the kernel trick.

## Part 3: Comparison and Reflection

### Time complexity

Compare the above two methods (quadratic basis function, kernel method) in terms of time complexity.

In terms of time complexity, is using a kernel method suitable in this case? What are potential advantages of using a kernel method? Disadvantages? 

### Representation of the Feature Space

In this example, the size of the feature space was quadratic in the number of original features of the dataset. Consider the memory cost of representing the transformed data. Give an example of a kernel with a basis function that should not be attempted to be stored in memory.

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

My response
---------------
1. Time complexity

Quadratic basis function:

- The feature transformation $\phi(\mathbf{x})$ takes $O(d)$ time for each data point, where $d$ is the number of features. This results in $O(nd)$ time complexity for transforming the entire dataset.
- Computing the matrix $\Phi^T\Phi$ takes $O(nmd)$ time, where $m$ is the dimension of the transformed feature space.
- Solving the linear system $(\Phi^T\Phi + \lambda\mathbf{I})\mathbf{w} = \Phi^T\mathbf{t}$ takes $O(m^3)$ time.
- In total, the time complexity of the quadratic basis function method is $O(nd + nmd + m^3)$ -> $O(nmd)$

Kernel method:

- Computing the kernel matrix $\mathbf{K}$ takes $O(n^2d)$ time.
- Solving the linear system $(\mathbf{K} + \lambda\mathbf{I})\boldsymbol{\alpha} = \mathbf{t}$ takes $O(n^3)$ time.
- In total, the time complexity of the kernel method is $O(n^2d + n^3)$ -> $O(n^3)$

2. Representation of feature space: Example kernel with a basis fx that should not be stored in memory

The Radial Basis Function (RBF) kernel, also known as the Gaussian kernel, is an example of a kernel with a basis function that should not be attempted to be stored in memory. The RBF kernel is defined as:

$$k(x,y) = \exp(-\frac{||x-y||^2}{2\sigma^2})$$

where $\sigma$ is a positive parameter controlling the width of the Gaussian function.

The RBF kernel implicitly maps the input data points to an infinite-dimensional feature space, where the basis functions are an infinite set of radial basis functions. Attempting to explicitly represent the transformed data in this infinite-dimensional feature space would require **infinite memory**, which is infeasible.

Using the RBF kernel within kernel methods, such as kernelised support vector machines or kernel ridge regression, allows us to operate in the infinite-dimensional feature space implicitly through the **kernel trick** without explicitly storing the transformed data points. This enables capturing complex patterns and relationships in the data while keeping the memory cost manageable.

Keywords: infinite memory, kernel trick, kernelised support vector machine, kernel ridge regression



## Part 3.5: Other Kernels (Optional)

We have seen how to implement kerenlised regression for the quadratic basis function. There are of course many other kernels, such as 
- Guassian kernels
- Linear kernels
- 
Choose from the above or another

Depending on the kernel function you choose, you may find it easier to implement the function itself (k(x,y)) and use it in your implementation of "your_kernel", or you may be able to directly compute "your_kernel(X,Y)".

In [ ]:
def your_kernel_function(x, y):
    
    raise NotImplementedError # TODO, or directly do "your_kernel"
    
    return k

In [ ]:
def your_kernel(X, Y):
    
    raise NotImplementedError # TODO
    
    return K

Now for you to reimplement kernelised linear regression: your code here should not differ significantly from that which you used in Part 2.

In [ ]:
def your_kernelised_lr(X, l, split_rate):
    """Return RMSE for the training set and the test set
    
    Parameters
    ------------------------------------------------------
    X: numpy matrix, whole dataset 
    l: float, regularisation parameter
    split_rate: int, the percent of training dataset
    
    Returns
    ------------------------------------------------------
    rmse_train: float, RMSE for training set
    rmse_test: float, RMSE for testing set
    """
    
    X_train, t_train, X_test, t_test = split_data(X, split_rate)
    N_train = X_train.shape[0]
    N_test = X_test.shape[0]
    
    raise NotImplementedError # TODO
    
    # Training set error
    t_train_pred = None # TODO
    rmse_train = np.linalg.norm(t_train_pred - t_train) / np.sqrt(N_train)

    # Test set error
    t_test_pred = None # TODO
    rmse_test = np.linalg.norm(t_test_pred - t_test) / np.sqrt(N_test)

    return rmse_train, rmse_test

train_rmse, test_rmse = your_kernelised_lr(data, 1.1, 0.8)
print("Kernelised Regression with your Kernel Function: RMSE with regularisation: Train {:.6f}, Test {:.6f}".format(train_rmse, test_rmse))

Why did you choose the kernel function that you did? How did it perform relative to the quadratic kernel? In what situation (type of dataset, properties of the data or distribution) might your chosen kernel function be advantagous compared to the quadratic kernel? 

## Textbook Questions (Optional)
These questions are hand picked to both be of reasonable difficulty and demonstrate what you are expected to be able to solve. The questions are labelled in Bishop as either $\star$, $\star\star$, or $\star\star\star$ to rate its difficulty.

- **Question 6.1**: Understand dual formulation (Difficulty $\star\star$, simple algebraic derivation)
- **Question 6.2**: Dual formulation for perceptron learning algorithm (Difficulty $\star\star$, simple algebraic derivation)
- **Question 6.11**: You may want to use Taylor expansion to represent term $\exp(\mathbf{x}^T\mathbf{x}'/2\sigma^2)$ (Difficulty $\star$)
- **Question 6.12**: To prove $k\left(A_{1}, A_{2}\right)=2^{\left|A_{1} \cap A_{2}\right|}=\phi\left(A_{1}\right)^{T} \phi\left(A_{2}\right)$. (Difficulty $\star\star$)
- **Question 6.13**: Use chain rule to represent $g(\varphi(\theta), x)$ (Difficulty $\star$)
- **Question 7.6**: (Difficulty $\star$, simple algebraic derivation)
- **Question 7.7**: (Difficulty $\star$, simple algebraic derivation)